In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summary/news_summary(1).csv


In [1]:
!pip install transformers  accelerate sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM




In [4]:

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [5]:
input_csv_file = "/kaggle/input/news-summary/news_summary(1).csv"  # Replace with your dataset file
output_csv_file = "output.csv"

In [6]:
data = pd.read_csv(input_csv_file, encoding='ISO-8859-1')  # Try 'latin1' or 'utf-16' if 'ISO-8859-1' doesn't work

In [7]:
def summarize_document(document):
    inputs = tokenizer.encode("summarize: " + document, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [8]:
generated_summaries = []

for index, row in data.iterrows():
    document = row['text']  # Change to 'text' or 'content' based on your dataset
    generated_summary = summarize_document(document)
    generated_summaries.append(generated_summary)

    # Print the generated summary for this row
    print(f"Generated Summary for Row {index + 1}:\n{generated_summary}\n")

Generated Summary for Row 1:
The Union Territory of Daman and Diu has revoked a decision that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on the occasion of Rakshabandhan.

Generated Summary for Row 2:
Malaika Arora has slammed an Instagram user who trolled her for "divorcing a rich man" and "having fun with the alimony" after he trolled her for "divorcing a rich man".

Generated Summary for Row 3:
A medical institute in the Indian state of Bihar has changed its Marital Declaration Form to 'Unmarried' after a public outcry over a change in the wording of a Marital Declaration Form for new recruits.

Generated Summary for Row 4:
Pakistani security forces have killed a Kashmiri militant who was captured by security forces in the north-eastern city of Jammu and Kashmir, the Pakistani state news agency AP reports, adding that he will not surrender to the militants.

Generated Summary for Row 5:
Hotels in India are to train staff to s

TypeError: can only concatenate str (not "float") to str

In [9]:
# Create a new DataFrame with the generated summaries
output_data = pd.DataFrame({'generated_summary': generated_summaries})

# Save the DataFrame to a new CSV file
output_data.to_csv(output_csv_file, index=False)

In [10]:
! pip install evaluate
! pip install rouge_score  
! pip install Bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6c3eead187d5c64401c6cfa109de1709dddf6e57d57ed32eb39d978f2f8c8c70
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065

In [11]:
import pandas as pd
from evaluate import load

# Load the ROUGE score model
rouge = load('rouge')

# Extract actual and predicted values from the DataFrame
n = 10  # Set the number of top rows you want to use
predictions = output_data["generated_summary"][:n].tolist()
references = data["text"][:n].tolist()

# Compute ROUGE scores for each pair of actual and predicted values
results = rouge.compute(predictions=predictions, references=references)

# Save the ROUGE scores to a new CSV file
output_df = pd.DataFrame({
    "Actual": references,
    "Predicted": predictions,
    "ROUGE-1": results['rouge1'],
    "ROUGE-2": results['rouge2'],
    "ROUGE-L": results['rougeL'],
    "ROUGE-Lsum": results['rougeLsum']
})

output_df.to_csv("rouge_scores_output.csv", index=False)

# Print the ROUGE scores
print("ROUGE Scores:")
print("ROUGE-1:", results['rouge1'])
print("ROUGE-2:", results['rouge2'])
print("ROUGE-L:", results['rougeL'])
print("ROUGE-Lsum:", results['rougeLsum'])


ROUGE Scores:
ROUGE-1: 0.49845118143424205
ROUGE-2: 0.3200797356030736
ROUGE-L: 0.4156975168477487
ROUGE-Lsum: 0.414580782462599


In [12]:
import pandas as pd
from evaluate import load

# Load the BERT score model
bertscore = load("bertscore")

# Extract actual and predicted values from the DataFrame
n = 10  # Set the number of top rows you want to use
predictions = output_data["generated_summary"][:n].tolist()
references = data["text"][:n].tolist()

# Compute BERT scores for each pair of actual and predicted values
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

# Extract individual BERT scores (precision, recall, F1) for each pair
precision = results["precision"]
recall = results["recall"]
f1 = results["f1"]

# Create a new DataFrame to store the BERT scores
output_df = pd.DataFrame({
    "Actual": references,
    "Predicted": predictions,
    "Precision": precision,
    "Recall": recall,
    "F1": f1
})

# Save the BERT scores to a new CSV file
output_df.to_csv("bert_scores_output.csv", index=False)

# Calculate the final cumulative BERT scores
final_precision = sum(precision) / len(precision)
final_recall = sum(recall) / len(recall)
final_f1 = sum(f1) / len(f1)

# Print the final cumulative BERT scores
print("Final Cumulative BERT Scores:")
print("Precision:", final_precision)
print("Recall:", final_recall)
print("F1 Score:", final_f1)


Final Cumulative BERT Scores:
Precision: 0.8775716483592987
Recall: 0.8205654621124268
F1 Score: 0.8475975394248962


In [13]:
import pandas as pd
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu

# Load BLEU score function from NLTK
def compute_bleu_score(prediction, references):
    return sentence_bleu(references, prediction)

# Extract actual and predicted values from the DataFrame
n = 10  # Set the number of top rows you want to use
predictions = output_data["generated_summary"][:n].tolist()
references_list = data["text"][:n].apply(lambda x: [reference.strip() for reference in x.split(',')]).tolist()

# Compute BLEU scores for each pair of actual and predicted values
bleu_scores = [compute_bleu_score(prediction, references) for prediction, references in zip(predictions, references_list)]

# Create a new DataFrame to store the BLEU scores
output_df = pd.DataFrame({
    "Actual": data["text"][:n],
    "Predicted": predictions,
    "BLEU Score": bleu_scores
})

# Save the BLEU scores to a new CSV file
output_df.to_csv("bleu_scores_output.csv", index=False)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Print the average BLEU score
print("Average BLEU Score:", average_bleu_score)


Average BLEU Score: 0.5585676198514739
